In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

import re

In [2]:
page = requests.get("https://www.amazon.com.br/b/?node=16339926011&pf_rd_r=8THN8DZ0V5AS6M9T0J6C&pf_rd_p=10f1e499-203c-47c7-b617-3cbf30b8c869&pd_rd_r=11acf4ef-154f-4517-b0bc-20552b6726d3&pd_rd_w=xY9Y5&pd_rd_wg=Tp1YO&ref_=pd_gw_unk")
page

<Response [200]>

We can print out the HTML content of the page using the content property

In [ ]:
page.content

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
cat = soup.find(class_="sl-sobe-carousel-viewport-row")
print(cat.prettify())

In [14]:
list_link = list()
names_link = list()
for link in cat.findAll('a'):
  list_link.append(link.get('href'))
  
type_product = [pt.get_text() for pt in cat.select('.sl-sobe-carousel-sub-card-title')]

print("We found", len(list_link), 'links')  
print("These links are from of the products:", type_product)  


We found 21 links
These links are from of the products: ['Monitores', 'Notebooks', 'Cadeiras gamer', 'Headsets', 'Mouses', 'Teclados', 'Webcams', 'Mousepads', 'Armazenamento interno', 'Placas de vídeo', 'Impressoras e acessórios', 'Armazenamento externo', 'Conexão e rede', 'Tablets', 'Suportes', 'Mochilas, capas e estojos', 'Placas-mãe', 'Computadores de mesa', 'Memórias RAM', 'Processadores', 'No-breaks']


So the next step it's to go for each those link where each represent the category's of the Computers and Informatics products.

In [ ]:
# We try the first one, to check if everything it's "OK".
link_test = "https://www.amazon.com.br/{}".format(list_link[0])
page_test = requests.get(link_test)
page_test_soup = BeautifulSoup(page_test.content, 'html.parser')
#print(page_test_soup.prettify())


Before we make someting more general, we need to know what's the information that we want to get from each category. To make this more simple we will get each product's information from in the 'Best sellers' (Mais vendidos) section.

In [19]:
# Continuing...
link_BS = "https://www.amazon.com.br{}".format(page_test_soup.find(class_ = 'a-link-normal').get('href'))
page_BS = requests.get(link_BS)
page_BS_soup = BeautifulSoup(page_BS.content, 'html.parser')
#Removing a specific class
page_BS_soup.find('div',class_='zg-more-list-types-container').decompose()

names_product = [pt.get_text() for pt in page_BS_soup.select("._cDEzb_p13n-sc-css-line-clamp-3_g3dy1")]
number_evaluation = [pt.get_text() for pt in page_BS_soup.select(".a-size-small")]
avg_evaluation = [pt.get_text() for pt in page_BS_soup.select(".a-icon-alt")]
price_product = [pt.get_text() for pt in page_BS_soup.select(".a-size-base")]


df = pd.DataFrame({
    "type_product": type_product[0],
    "names_product": names_product,
    "number_evaluation": number_evaluation,
    "avg_evaluation": avg_evaluation,
    "price_product":price_product
})

df


,type_product,names_product,number_evaluation,avg_evaluation,price_product
0,Monitores,"Monitor AOC 21.5"" VGA e HDMI e painel VA, pret...",818,"4,8 de 5 estrelas","R$ 749,00"
1,Monitores,"Monitor Gamer AOC Hero 27"" 144Hz IPS 1ms G-Syn...",2.089,"4,7 de 5 estrelas","R$ 1.499,00"
2,Monitores,"Monitor Gamer AOC HERO Z 240Hz 0, 5ms IPS G-Sy...",150,"4,7 de 5 estrelas","R$ 1.751,00"
3,Monitores,Monitor para PC Full HD UltraWide LG LED IPS 2...,3.128,"4,8 de 5 estrelas","R$ 1.499,00"
4,Monitores,"Monitor LG UltraGear 24GN600-24"" IPS Full HD, ...",440,"4,7 de 5 estrelas","R$ 1.379,00"
5,Monitores,"Monitor AOC 23,8""",1.145,"4,8 de 5 estrelas","R$ 809,00"
6,Monitores,"Monitor Gamer Mancer Horizon Z27, 27 Pol. Va, ...",9,"4,8 de 5 estrelas","R$ 881,98"
7,Monitores,"Monitor Gamer AOC SNIPER 27"" 75Hz IPS 1ms",632,"4,7 de 5 estrelas","R$ 1.049,00"
8,Monitores,"Monitor Gamer AOC Hero 24"" 144Hz IPS 1ms AMD F...",921,"4,7 de 5 estrelas","R$ 1.329,99"
9,Monitores,"Monitor Gamer AOC LEGEND 27 240Hz 0,5ms, Preta...",613,"4,8 de 5 estrelas","R$ 1.899,00"


## The next problem it is with exist more than one page to search the prizes, so we need to add a specific code for this situation.
